### Ref.
https://qiita.com/yasunori/items/31a23eb259482e4824e2

## word segmentation

In [3]:
# -*- coding: utf-8 -*-
import MeCab
mecab = MeCab.Tagger('mecabrc')

In [4]:
def tokenize(text):
    '''
    とりあえず形態素解析して名詞だけ取り出す感じにしてる
    '''
    node = mecab.parseToNode(text)
    while node:
        if node.feature.split(',')[0] == '名詞':
            yield node.surface.lower()
        node = node.next


def get_words(contents):
    '''
    記事群のdictについて、形態素解析してリストにして返す
    '''
    ret = []
    for k, content in contents.items():
        ret.append(get_words_main(content))
    return ret


def get_words_main(content):
    '''
    一つの記事を形態素解析して返す
    '''
    return [token for token in tokenize(content)]

In [5]:
# 2記事の一部だけ取り出しました
# 1つめがITライフハック、2つめが独女通信の記事です。
words = get_words({'it-life-hack-001.txt': 'アナタはまだブラウザのブックマーク？　ブックマーク管理はライフリストがオススメ 最近ネットサーフィンをする際にもっぱら利用しているのが「ライフリスト」というサイトだ。この「ライフリスト」は、ひとことで言うと自分専用のブックマークサイトである。というよりブラウザのスタートページにするとブラウザのブックマーク管理が不要になる便利なサイトなのである。', 'dokujo-tsushin-001.txt': 'たとえば、馴れ馴れしく近づいてくるチャラ男、クールを装って迫ってくるエロエロ既婚男性etc…に対し「下心、見え見え〜」と思ったことはないだろうか？ “下心”と一言で言うと、特に男性が女性のからだを目的に執拗に口説くなど、イヤらしい言葉に聞こえてしまう。実際、辞書で「下心」の意味を調べてみると、心の底で考えていること。かねて心に期すること、かねてのたくらみ。特に、わるだくみ。（広辞苑より）という意味があるのだから仕方がないのかもしれない。'})
print(words)

[['チャラ', '男', 'クール', 'エロ', 'エロ', '既婚', '男性', 'etc', '下心', '見え', '見え', 'こと', '下心', '一言', '男性', '女性', 'からだ', '目的', '執拗', 'イヤ', '言葉', '辞書', '下心', '意味', '心', '底', 'こと', '心', 'こと', 'わる', 'くみ', '広辞苑', '意味', 'の', '仕方', 'の'], ['アナタ', 'ブラウザ', 'ブック', 'マーク', 'ブック', 'マーク', '管理', 'ライフ', 'リスト', 'オススメ', '最近', 'ネット', 'サーフィン', '際', '利用', 'の', 'ライフ', 'リスト', 'サイト', 'ライフ', 'リスト', 'ひとこと', '自分', '専用', 'ブックマークサイト', 'ブラウザ', 'スタート', 'ページ', 'ブラウザ', 'ブック', 'マーク', '管理', '不要', '便利', 'サイト', 'の']]


## Dictionary

In [6]:
from gensim import corpora

In [7]:
# words はさっきの単語リスト
dictionary = corpora.Dictionary(words)
print(dictionary.token2id)

{'自分': 46, 'アナタ': 27, 'くみ': 2, 'エロ': 7, 'こと': 3, 'イヤ': 6, 'からだ': 1, 'オススメ': 28, 'ひとこと': 26, '執拗': 13, 'etc': 0, 'ライフ': 38, '見え': 23, '広辞苑': 15, 'サーフィン': 30, '下心': 11, 'リスト': 39, 'ブックマークサイト': 34, '男性': 21, '仕方': 12, '便利': 41, 'の': 4, '最近': 44, '既婚': 19, 'ネット': 32, '言葉': 24, '管理': 45, 'ブック': 33, 'ブラウザ': 35, 'わる': 5, 'チャラ': 9, '底': 16, '一言': 10, '男': 20, 'サイト': 29, '専用': 43, 'クール': 8, '不要': 40, '女性': 14, '意味': 18, 'ページ': 36, 'マーク': 37, '辞書': 25, 'スタート': 31, '利用': 42, '目的': 22, '際': 47, '心': 17}


In [17]:
dictionary.filter_extremes(no_below=20, no_above=0.3)
# no_berow: 使われてる文章がno_berow個以下の単語無視
# no_above: 使われてる文章の割合がno_above以上の場合無視

In [8]:
dictionary.save_as_text('livedoordic.txt')

In [9]:
dictionary = corpora.Dictionary.load_from_text('livedoordic.txt')

## Vector

In [12]:
from gensim import corpora, matutils

# さっきITライフハックの1記事を形態素解析かけて名詞取り出したやつ
words = ['アナタ', 'ブラウザ', 'ブック', 'マーク', 'ブック', 'マーク', '管理', 'ライフ', 'リスト', 'オススメ', '最近', 'ネット', 'サーフィン', '際', '利用', 'の', 'ライフ', 'リスト', 'サイト', 'ライフ', 'リスト', 'ひとこと', '自分', '専用', 'ブックマークサイト', 'ブラウザ', 'スタート', 'ページ', 'ブラウザ', 'ブック', 'マーク', '管理', '不要', '便利', 'サイト', 'の']

tmp = dictionary.doc2bow(words)
dense = list(matutils.corpus2dense([tmp], num_terms=len(dictionary)).T[0])
print(dense)

[0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 3.0, 1.0, 3.0, 1.0, 3.0, 3.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0]


## Machine Learning

In [15]:
from sklearn.ensemble import RandomForestClassifier

# 1個目がITライフハックのある記事、 2個目が独女通信のある記事の特徴ベクトル
data_train = [[0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 3.0, 1.0, 3.0, 1.0, 3.0, 3.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0],[2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 3.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0]]

# 正解のラベル
label_train = [1,0]  # 1: ITライフハック、 0: 独女通信


estimator = RandomForestClassifier()

# 学習させる
estimator.fit(data_train, label_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [16]:
# 予測
label_predict = estimator.predict(data_train)
print(label_predict)

[1 0]
